In [366]:
import googlemaps
import folium
import pandas as pd
import numpy as np
import heapq
from haversine import haversine #거리구하기
from sklearn.preprocessing import minmax_scale #정규화

##1. data 불러오기
total_df = pd.read_csv("../all_df.csv", encoding = "EUC-KR" )
df = total_df.rename(index=str, columns={"Unnamed: 0" : "ID"})
df = df.drop(columns = ['경유여부','거리'])

##2. 출발지, 도착지
sp = (37.506059, 127.036863) #출발지 37.506059, 127.036863
ep = (37.509122, 127.043816) #도착지 37.509122, 127.043816

#4.중복값 제거
df = df.join( df.groupby("명칭")["ID"].count(), on = '명칭', rsuffix = '_cnt')
df = df[~(df.ID_cnt > 1)].sort_values(by = 'ID')
df = df.drop(columns = ['ID_cnt'])

#밝기 max
l_max = df['총밝기'].max()

# 3.table에 SP, EP정보 저장
df = df.append(df.from_dict({'ID' : -1,'point_ID':'Point_SP', '위도':sp[0], '경도':sp[1], '밝기':-1
                             , '분류':'출발지', '명칭':'출발지', '총밝기' : -1 }, orient = 'index').T)

df = df.append(df.from_dict({'ID' : len(total_df), 'point_ID':'Point_EP', '위도':ep[0], '경도':ep[1], '밝기': 999
                             , '분류':'도착지', '명칭':'도착지', '총밝기' : 999 }, orient = 'index').T)



df = df.sort_values(by='ID').reset_index(drop=True)
df1 = df
df1['ID'] = df1.index
df1.tail(4)


,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
1444,1444,Point_2087,127.043,역삼동 700-13,25,CCTV,37.505,70
1445,1445,Point_2088,127.037,역삼동 652-5,25,CCTV,37.5082,50
1446,1446,Point_2089,127.034,역삼동 735-18,25,CCTV,37.4979,110
1447,1447,Point_EP,127.044,도착지,999,도착지,37.5091,999


In [186]:
#총 거리
d_m = haversine(sp, ep, unit='m') # in meters
d_km = haversine(sp, ep,  unit='m')  # in kilometers

#block단위 
# 경유지가 총 3개까지만 가능하니까 3개의 반경으로 나누자. 700 이상인 경우는 api다시 받기 (추후에)
dvd = d_m / 4
print( "단위거리(dvd) : ",  dvd, ", 총거리(d_m) : " , d_m)

단위거리(dvd) :  175.38377731330854 , 총거리(d_m) :  701.5351092532342


# 길찾기 구현 시작 (A* 알고리즘)
> 참고 사이트 : https://codeday.me/ko/qa/20190421/370160.html
### 1. 초기화

In [444]:
#!pip install geopy
import csv, geopy.distance, collections, heapq
##name
Location = collections.namedtuple("Location", "ID point_ID 분류 명칭 위도 경도 밝기 총밝기".split())
data = {}

for i in range(0,len(df1)):
        data[i] = Location(  df1.loc[i,"ID"]
                           , df1.loc[i,"point_ID"]
                           , df1.loc[i,"분류"]
                           , df1.loc[i,"명칭"]
                           , df1.loc[i,"위도"]
                           , df1.loc[i,"경도"]
                           , df1.loc[i,"밝기"]
                           , df1.loc[i,"총밝기"])
        
print( type(data) , len(data))
#sorted(data.values(), key=lambda x: calcH(0,len(data)-1))[0:4]

startIndex = 0                                                     #출발지
endIndex = len(data) - 1                                           #도착지
h = calcH(startIndex, endIndex)                                    #도착지까지의 총거리

#heap으로 하면 좋겠지만, sclaedF 를 구하기 위해 dataFrame으로 
openlist = pd.DataFrame(columns = "ID F G H L parentID".split() )
openlist.loc[0] = [startIndex, h, 0, h, 0 , None]                #openlist 초기화
closedlist = {}      # map visited nodes to parent #LinkedList
openlist

<class 'dict'> 1448


,ID,F,G,H,L,parentID
0,0.0,701.535109,0.0,701.535109,0.0,NaN


### 2. 함수 정의 

In [269]:
#거리 구하기
def calcH(start_index, end_index):
    distance = haversine([data[start_index].위도, data[start_index].경도] , [data[end_index].위도, data[end_index].경도] ,  unit='m') 
    return distance

#주변 점들 구하기
def getneighbors(center_index, n=10):
    return sorted(data.values(), key=lambda x: calcH(center_index, x.ID))[1:n+1]

#경로 추출하기
def getParent(closedlist, index):
    path = []
    while index is not None:
        path.append(index)
        index = closedlist.get(index, None)
    return [data[i] for i in path[::-1]]


# 가중치 구하기
#1. F구하기 



def get_f(center_node, item ):
    g = center_node.G + calcH(center_node.ID, item.ID)
    h = calcH(item.ID, endIndex)
    f = g + h   
    #f = get_scaled_f(item)
    #w = get_w()
    return f

#2. Scaled_F : 밝기 0~100 에 맞추기
# 근데 상대적으로 계속 변하는 값이라서 흠.. 이 구조가 어울리지 않는가?? 으악!!!!!!!!!!
# 그럼 openlist자체가 어울리지 않는 건가??
## 그럼 새로운 F가 들어올때마다 openlist의 모든 F는 갱신 tmp를 만들어야하나 

def get_scaled_f(item, f, g, h):
    openlist_tmp = copy.deepcopy(openlist)
    openlist_tmp.loc[len(openlist_tmp)] = [int(item.ID), f, g, h, int(item.총밝기), center_node.parentID ]
    openlist_tmp['Scaled_F'] = l_max - ( minmax_scale(openlist_tmp['F'], axis=0, copy=True, feature_range =(0,l_max)).astype(int))
    openlist_tmp.sort_values(by='Scaled_F')
    return Scaled_F

#3. 밝기 이용한 가중치 추가 W(weight = 4F(거리) + 6L(밝기))
def get_w(df) : 
    w = np.array(df.apply(lambda x : (x['총밝기'] * 45 ) + ( x['scaled_F'] * 100) , axis = 1)).reshape(-1,1)
    w = minmax_scale(w, copy=True, feature_range =(0,l_max))
    return w


In [270]:
# while len(openlist) >= 1:
#     fin = []
#     # 우선순위큐, 가장 점수높은 후보 꺼내서 중심점으로 만들기
#     _, center_node = heapq.heappop(openlist)
#     print(center_node)
    
#     # 이미 방문한 지점인지 검사
#     if center_node.ID in closedlist:
#         continue
#     closedlist[center_node.ID] = center_node.parentID

#     # 만약 도착지라면 종료하고, 경로 추출하기
#     if center_node.ID == endIndex:
#         #print("Complete")
#         for p in getParent(closedlist, center_node.ID):
#             #print(p)
#             fin.append(p)
#         break
        
    # 주변 점들을 구하고, 점들마다 예상 가중치를 구해서 비교
    for item in getneighbors(center_node.ID):
        f = get_f(center_node, item )
        heapq.heappush(openlist, (f, Node(item.ID, f, g, h, center_node.ID)))
        
        
        #현재점을 거쳐서 도착지까지 g 계산
#         for i in range(0, len(openlist)):
#             ids.append(openlist[i][1].ID)
#         if other.ID in ids

        ####openList에 추가###
        #새롭게 계산한 예상되는 F  < 이미 openlist에 있는 F  : 새롭게 예상한 아이가 더 좋은 경로 -> parentID 갱신
        # but, 중복되게 들어가더라도 항상 가장 가능성 높은 아이가 자동으로 나오므로 굳이 갱신할 필요가없다.
        #새롭게 계산한 예상되는 F  > 이미 openlist에 있는 F : 그대로 유지  


for item in getneighbors(center_node.ID):
    f = get_f(center_node, item )
    heapq.heappush(openlist, (f, Node(item.ID, f, g, h, center_node.ID)))


In [415]:
f = 650
g = 250
h = 400
ID = item.ID

openlist = pd.DataFrame(columns = "ID F G H L parentID".split() )
openlist.loc[0] = [startIndex, h, 0, h, 0, None]  
print(item)
openlist_tmp = openlist

openlist_tmp

X_MinMax_scaled =X_MinMax_scaled = l_max - minmax_scale(openlist_tmp['F'], axis=0, copy=True, feature_range =(0,l_max))


openlist_tmp

Location(ID=802, point_ID='Point_1320', 분류='보안등', 명칭='역삼1동 660-28', 위도=37.5060188842, 경도=127.03742662100001, 밝기=5, 총밝기=70)


,ID,F,G,H,L,parentID
0,0.0,400.0,0.0,400.0,0.0,NaN


In [416]:
for i in range(0, 10) :
    f += 20
    g += 10
    h += 10
    ID += 1
    openlist_tmp.loc[len(openlist)] = [int(ID), f, g, h, item.총밝기, center_node.ID ]
    
openlist_2 = openlist_tmp
l_max

185

In [445]:
def get_f(center_node, item ):
    g = center_node.G + calcH(center_node.ID, item.ID)
    h = calcH(item.ID, endIndex)
    f = g + h   
    sf = get_scaled_f(item,f,g,h)
 #   w = get_w()
    return f

def get_scaled_f(item, f, g, h):
    openlist_tmp = pd.DataFrame()
    openlist_tmp = openlist
    openlist_tmp['잉'] 
#     openlist_tmp.loc[len(openlist)] = [int(item.ID), f, g, h, int(item.총밝기), center_node.parentID ]
    
#     openlist_tmp['Scaled_F'] = l_max - ( minmax_scale(openlist_tmp['F'], axis=0, copy=True, feature_range =(0,l_max)).astype(int))
#     openlist_tmp.sort_values(by='Scaled_F')
#     return openlist_tmp['Scaled_F']

#3. 밝기 이용한 가중치 추가 W(weight = 4F(거리) + 6L(밝기))
def get_w(df) : 
    w = np.array(df.apply(lambda x : (x['총밝기'] * 45 ) + ( x['scaled_F'] * 100) , axis = 1)).reshape(-1,1)
    w = minmax_scale(w, copy=True, feature_range =(0,l_max))
    return w

In [454]:
import copy
#heap으로 하면 좋겠지만, sclaedF 를 구하기 위해 dataFrame으로 
openlist = pd.DataFrame(columns = "ID F G H L parentID".split() )
openlist.loc[0] = [startIndex, h, 0, h, 0 , None]    

items = getneighbors(center_node.ID)
print(center_node , "\n ", item, len(items))

#openlist

openlist_tmp =copy.deepcopy(openlist)
#openlist_tmp = openlist
openlist_tmp['hi'] = [0] 
openlist


Node(ID=0, F=701.5351092532342, G=0, H=701.5351092532342, parentID=None) 
  Location(ID=802, point_ID='Point_1320', 분류='보안등', 명칭='역삼1동 660-28', 위도=37.5060188842, 경도=127.03742662100001, 밝기=5, 총밝기=70) 10


,ID,F,G,H,L,parentID
0,0.0,701.535109,0.0,701.535109,0.0,NaN


In [375]:
# 역삼동 중심 = 역삼역 /  좌표 : 37.500742, 127.036891
to_map1 = folium.Map(location= sp, zoom_start=18)
a = closed_list
b 
# for n in range(0, len(a)) :|
#     if a['분류'][n] == '편의점' :
#         folium.Marker([a['위도'][n], a['경도'][n]], popup= a['분류'][n], icon=folium.Icon(color='green')).add_to(to_map1)
#     elif a['분류'][n] == 'CCTV' : 
#         folium.CircleMarker([a['위도'][n], a['경도'][n]], popup= a['point_ID'][n] + a['명칭'][n], radius = a['밝기'][n] + 1).add_to(to_map1)        
#     elif a['분류'][n] == '보안등' : 
#         folium.CircleMarker([a['위도'][n], a['경도'][n]], popup= a['point_ID'][n] + a['명칭'][n], radius = a['밝기'][n] + 1).add_to(to_map1)  
#     else : 
#         folium.Marker([a['위도'][n], a['경도'][n]], popup= a['분류'][n], icon=folium.Icon(color='black')).add_to(to_map1)        


for n in range(0, len(a)) :
      folium.CircleMarker([a[n][0], a[n][1]], popup= str(closed_list_seq[n]), radius = 3).add_to(to_map1)        

# 구분하기 쉽게 스타팅 지역은 빨간색 마커로 표시        
folium.Marker(sp, popup= 'SP', icon=folium.Icon(color='red')).add_to(to_map1)
folium.Marker(ep, popup= 'EP', icon=folium.Icon(color='black')).add_to(to_map1)
to_map1


NameError: name 'closed_list' is not defined

In [208]:
df[df['point_ID'] == 'Point_248' ]
df[df['명칭'].str.contains('목화') ]

#closed_list_id

,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
1717,1795,Point_1795,127.043,GS25 역삼목화점,30,편의점,37.5089,75


In [211]:
dd = total_df[ total_df['point_ID'].isin(closed_list_id) ].sort_values('총밝기', ascending = False)
dd

,Unnamed: 0,point_ID,거리,밝기,총밝기,분류,명칭,위도,경도,경유여부
1789,1789,Point_1789,0,30,110,편의점,CU 역삼에스원점,37.506208,127.037951,0
1834,1834,Point_1834,0,25,110,CCTV,역삼동 655-13,37.506711,127.037181,0
1437,1437,Point_1437,0,5,110,보안등,역삼1동 661-3,37.505838,127.038051,0
1379,1379,Point_1379,0,5,105,보안등,역삼1동 656-12,37.506415,127.037943,0
1323,1323,Point_1323,0,5,105,보안등,역삼1동 655-13,37.506913,127.037214,0
1438,1438,Point_1438,0,5,105,보안등,역삼1동 660-33,37.505781,127.037757,0
1637,1637,Point_1637,0,30,100,편의점,GS25 역삼충현점,37.505715,127.037723,0
1316,1316,Point_1316,0,5,90,보안등,역삼1동 656-5,37.506599,127.037603,0
1503,1503,Point_1503,0,5,85,보안등,역삼1동 661-1,37.506062,127.037942,0
1317,1317,Point_1317,0,5,85,보안등,역삼1동 657-40,37.506480,127.037314,0


In [210]:
df[df['point_ID'] == 'Point_253' ]

,ID,point_ID,경도,명칭,밝기,분류,위도,총밝기
227,253,Point_253,127.043,역삼1동 684-27,5,보안등,37.5089,75
